- TA-Lib Examples https://mrjbq7.github.io/ta-lib/abstract.html Technical analysis library
- Zipline https://www.zipline.io/index.html backtesting library
- Pandas Datareader https://pydata.github.io/pandas-datareader/ - read data into pandas
- Alphalens https://github.com/quantopian/alphalens
- Pyfolio https://github.com/quantopian/pyfolio - Library for performance & risk analysis of financial portfolios
- pySecMaster https://github.com/camisatx/pySecMaster - Automated framework to store and maintain financial data.


Trading system:
- buy signal is three green flags
 - SMA crossover
 - STOCH crossover
 - MACH crossover
- sell signal is one red flag
 - sell when any one flag reverses

In [1]:
from datetime import datetime
import pandas as pd
import pandas_datareader as web
import pandas_ta
# import talib
# import talib.abstract as ta
import helpers

ModuleNotFoundError: No module named 'pandas_datareader'

In [2]:
def get_ticker_price_history(ticker, start, end, source='yahoo'):
    df = web.DataReader(ticker, source, start, end)
    df['ticker'] = ticker
    # df = df.set_index('ticker')
    return df

In [3]:
start = datetime(2016, 1, 1)
end = datetime.today()
tickers = ['SPHD', 'SPYD', 'F', 'NRZ']

In [4]:
# put portfolio price history in a df
df = pd.DataFrame()
for ticker in tickers:
    tmp = get_ticker_price_history(ticker, start, end, source='yahoo')
    df = pd.concat([df, tmp], axis=0)

In [5]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,high,low,open,close,volume,adj_close,ticker
Date,,,,,,,
2016-01-04,33.049999,32.699001,32.970001,33.040001,289500.0,27.232388,SPHD
2016-01-05,33.380001,32.985001,33.110001,33.349998,158600.0,27.487906,SPHD
2016-01-06,33.200001,32.958000,33.000000,33.090000,289400.0,27.273609,SPHD
2016-01-07,32.910000,32.425999,32.700001,32.470001,282600.0,26.762592,SPHD
2016-01-08,32.669998,32.220001,32.660000,32.299999,265000.0,26.622465,SPHD


In [11]:
df.to_csv('stock_prices.csv', index=False)

In [6]:
# adjust prices of high, low, open
df['adj_pct'] = df.adj_close / df.close
for item in ['high', 'low', 'open']:
    name = 'adj_' + item
    df[name] = df[item] * df.adj_pct

In [7]:
# Make df a multi-index
df = df.set_index(['ticker', df.index]).sort_index()

In [8]:
def ema(df, ticker, timeperiod, field='adj_close'):
    return ta.EMA(df.loc[ticker, :], timeperiod, field)

def sma(df, ticker, timeperiod, field='adj_close'):
    return ta.SMA(df.loc[ticker, :], timeperiod=timeperiod)

def macd(df, ticker, fast=12, slow=26, signal=9, field='adj_close'):
    '''
    The Moving Average Convergence Divergence (MACD) is the difference between two
    Exponential Moving Averages. The Signal line is an Exponential Moving Average of the MACD.

    '''
    close = df.loc[ticker, :]['adj_close']
    return (ta.MACD(close, fastperiod=fast, slowperiod=slow, signalperiod=signal))

def slowstochastic(df, ticker, fastk_periods=3, slowk_periods=14, slowd_periods=14):
    '''
    Stochastics are used to show when a stock has moved into an overbought or oversold position.
    The Stochastic Oscillator measures where the close is in relation to the recent trading range. 
    The values range from zero to 100. %D values over 75 indicate an overbought condition; 
    values under 25 indicate an oversold condition. 
    '''
    high  = df.loc[ticker, :]['adj_high']
    low   = df.loc[ticker, :]['adj_low']
    close = df.loc[ticker, :]['adj_close']
    slowk, slowd = ta.STOCH(high, low, close, fastk_period=fastk_periods, slowk_period=slowk_periods, 
                            slowk_matype=0, slowd_period=slowd_periods, slowd_matype=0)    
    return (slowk, slowd)
    
def faststochastic(df, ticker, fastk_periods=3, fastd_periods=3):
    high  = df.loc[ticker, :]['adj_high']
    low   = df.loc[ticker, :]['adj_low']
    close = df.loc[ticker, :]['adj_close']
    fastk, fastd = ta.STOCHF(high, low, close, fastk_period=fastk_periods,
                             fastd_period=fastd_periods, fastd_matype=0)
    return (fastk, fastd)

def stoch_oscillator():
    '''
    When the Fast %D crosses above the Slow %D, 
    it is a buy signal; when it crosses below, it is a sell signal.
    '''
    def pct_k(adj_close, timeperiods=14):
        minidx, maxidx = MINMAXINDEX(adj_close, timeperiod=timeperiods)
        return 100 * (adj_close - minidx) / (maxidx - minidx)

    def pct_d(adj_close, timeperiods=14):
        return ta.SMA(field=adj_close, timeperiod=timeperiods)
    
    k = pct_k()
    d = pct_d(k)
    return k, d



In [9]:
for ticker in tickers:
    macdtuple = macd(df, ticker, fast=12, slow=26, signal=9, field='adj_close')
    slowstoch = slowstochastic(df, ticker, fastk_periods=3, slowk_periods=14, slowd_periods=14)
    faststoch = faststochastic(df, ticker, fastk_periods=14, fastd_periods=14)
    
    df.loc[ticker, :]['sma30'] = sma(df, ticker, timeperiod=30)
    df.loc[ticker, :]['sma50'] = sma(df, ticker, timeperiod=50)
    df.loc[ticker, :]['ema30'] = ema(df, ticker, timeperiod=30)
    df.loc[ticker, :]['ema50'] = ema(df, ticker, timeperiod=50)
    df.loc[ticker, :]['macd'] = macdtuple[0]
    df.loc[ticker, :]['macdsignal'] = macdtuple[1]
    df.loc[ticker, :]['macdhist']  = macdtuple[2]
    df.loc[ticker, :]['slowd'] = slowstoch[1]
    df.loc[ticker, :]['fastd'] = faststoch[1]


In [10]:
from pprint import pprint
pprint(ta.Function('sma').info)

{'display_name': 'Simple Moving Average',
 'function_flags': ['Output scale same as input'],
 'group': 'Overlap Studies',
 'input_names': OrderedDict([('price', 'close')]),
 'name': 'SMA',
 'output_flags': OrderedDict([('real', ['Line'])]),
 'output_names': ['real'],
 'parameters': OrderedDict([('timeperiod', 30)])}


In [11]:
print(talib.get_functions())

['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'ADD', 'DIV', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'MULT', 'SUB', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'ADX', 'ADXR', 'APO', 'AROON', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH', 'STOCHF', 'STOCHRSI', 'TRIX', 'ULTOSC', 'WILLR', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MAVP', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR',

In [23]:
s = sma(df, 'F', timeperiod=30)
pd.concat([df, s.to_frame()], ignore_index=True)

,0,adj_close,adj_high,adj_low,adj_open,adj_pct,close,high,low,open,volume
0,NaN,11.050024,11.073753,10.876008,10.970925,0.790982,13.97,14.00,13.75,13.87,38618500.0
1,NaN,10.852281,11.073756,10.686174,11.050026,0.790983,13.72,14.00,13.51,13.97,50267500.0
2,NaN,10.369781,10.725724,10.322323,10.725724,0.790983,13.11,13.56,13.05,13.56,61285500.0
3,NaN,10.045479,10.314413,9.966381,10.203675,0.790983,12.70,13.04,12.60,12.90,57846700.0
4,NaN,9.918921,10.361871,9.887281,10.322322,0.790982,12.54,13.10,12.50,13.05,46199400.0
...,...,...,...,...,...,...,...,...,...,...,...
5950,6.970333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5951,6.970667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5952,6.960333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5953,6.940667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
